# Current CRM

In [2]:
import pandas as pd
import numpy as np
import datetime
import sys, os
import glob
import time
import fnmatch
import time
import xlwings as xw
start_time = time.time()

                                                   #OH
OH = pd.read_excel("OH.xlsx", dtype={'Item': str})
#print(OH.shape)
#OH.head()
#SUBSET FOR NL1 SUBINVENTORY
OH= OH[ ( (OH['Org'] == 'NL1') & 
         ( (OH['Sub'] == 'CMF1') | (OH['Sub'] == 'CFG1') | (OH['Sub'] == 'FIG1') | (OH['Sub'] == 'FWP1') | (OH['Sub'] == 'FWP2') 
         | (OH['Sub'] == 'OFS1') | (OH['Sub'] == 'PAQ1') | (OH['Sub'] == 'PAR1') | (OH['Sub'] == 'PAR2') | (OH['Sub'] == 'PTO2')
         | (OH['Sub'] == 'SRM1') | (OH['Sub'] == 'STG1') | (OH['Sub'] == 'STG2') | (OH['Sub'] == 'STG3') | (OH['Sub'] == 'STG8')
         | (OH['Sub'] == 'STG9') | (OH['Sub'] == 'SYS1') | (OH['Sub'] == 'SYS2') | (OH['Sub'] == 'SYS3') )) | 
       (OH['Org'] == 'SG1') | (OH['Org'] == 'TX2') | (OH['Org'] == 'SH2')]            
#Create SKU DC Key
OH["SKU DC"] = OH["Item"] + OH["Org"]
#OH.head()
#make OH PIVOT
OHPivot = pd.pivot_table(data = OH, index = ['SKU DC'], values = ['On-hand'], aggfunc = np.sum)
OHPivot.reset_index(inplace=True) #CREATE NEW INDEXES
#ADD ORG & ITEM COLUMNS
OHPivot['ITEM'] = OHPivot['SKU DC'].str[:-3]
OHPivot['ORG'] = OHPivot['SKU DC'].str.strip().str[-3:]
#REORDER COLUMNS AS WANTED
mid = OHPivot['SKU DC']
OHPivot.drop(labels=['SKU DC'], axis=1, inplace = True)
OHPivot.insert(3, 'SKU DC', mid)
mid = OHPivot['On-hand']
OHPivot.drop(labels=['On-hand'], axis=1, inplace = True)
OHPivot.insert(3, 'OH', mid)
#print(OHPivot.shape, 'OH tab created') # OHPivot to write as OH tab
#OHPivot.head()

#ASN
#a=fnmatch.filter(os.listdir('.'), 'ASNS*')
#a=a[0] 
#dfASN = pd.read_excel(a,dtype={'*Zebra PN': str},sheet_name = 'CRM',parse_dates=['*Planned Delivery Date.1'])
#dfASN['*Planned Delivery Date.1']= dfASN['*Planned Delivery Date.1'].dt.strftime('%m-%d-%Y')

                              #SRDC
b=fnmatch.filter(os.listdir('.'), 'SRDC*')
b=b[0] 
print('Reading ', b)
dfSRDCNL1 = pd.read_excel(b,dtype={'ITEM': str},sheet_name = 'SRDC_NL1,SG1,SH2',usecols="A,B,D,F")

dfSRDCTX = pd.read_excel(b,dtype={'ITEM': str},sheet_name = 'SRDC_TX2,TX3,TX4,TX5',usecols="A,B,D,F")
dfSRDC = pd.concat([dfSRDCNL1,dfSRDCTX], ignore_index=True)
#print(dfSRDC.shape, 'all SRDC')
dfSRDC = dfSRDC[(dfSRDC['ORDER_TYPE'] == 'Inter-Org Intransit')]
#print(dfSRDC.shape, 'filtered SRDC')

                            #SOURCING RULES
c=fnmatch.filter(os.listdir('.'), 'Sourcing Rules*')
c=c[0] 
print('Reading ', c)
dfsourcingrule  = pd.read_excel(c,dtype={'Item': str},sheet_name = 'Sourcing Rules')

dfsourcingrule = dfsourcingrule[(dfsourcingrule['Org'] != 'TX3')]
dfsourcingrule = dfsourcingrule.drop(['Inventory Org Name'], axis = 1)
#print(dfsourcingrule.shape, 'ITEMS 1 SHAPE')
dfsourcingrule['Item'] = dfsourcingrule['Item'].astype(str)

                                        #ABCD file
print('Reading ABCD')
ABCD = pd.read_csv("Product Family & ABCD Rating.csv", dtype={'Item': str,'Inventory Org Name': str})
ABCD['SKU DC'] = ABCD["Item"]+ABCD['Inventory Org Name'].str[:3]
ABCD.drop(labels=['Default Buyer','Product Family','Product Class','Inventory Org Name'], axis=1, inplace = True)
ABCD=ABCD.drop_duplicates(subset='SKU DC',keep='first') 

dfsourcingrule = pd.merge(dfsourcingrule, ABCD, on='SKU DC', how='left')
dfsourcingrule.rename(columns = {'Zebra ABCD Rating':'ABCD RATING'}, inplace = True)

dfsourcingrule['PLANNING CATEGORY'] = np.where(dfsourcingrule['ABCD RATING'] == 'UNASSIGN','Obsolete',
                                      np.where(dfsourcingrule['ABCD RATING'] == 'A','Super Core',
                                      np.where(dfsourcingrule['ABCD RATING'] == 'B','Core',
                                      np.where(dfsourcingrule['ABCD RATING'] == 'C','Non Core',
                                      np.where(dfsourcingrule['ABCD RATING'] == 'D','Slow Moving','NA' )))))

dfsourcingrule.drop(labels=['Item_y'], axis=1, inplace = True)
dfsourcingrule.rename(columns={'Item_x': 'Item'},inplace =True)
dfsourcingrule.drop(labels=['Product Family Description'], axis=1, inplace = True)

my_date = datetime.date.today()
year, week_num, day_of_week = my_date.isocalendar()
strings= 'VARIOUS WK' + str(week_num)
b=("%s.xlsx" % strings)
print('CRM file created')

Reading  Sourcing Rules WK 32.xlsx
Reading ABCD
CRM file created


# Try for ITEMS 2 report

In [3]:
dfsourcingrule  = pd.read_excel(c,dtype={'Item': str},sheet_name = 'Sourcing Rules')
dfsourcingrule = dfsourcingrule[(dfsourcingrule['Org'] != 'TX3') & (dfsourcingrule['Org'] != 'TX5')]
dfsourcingrule = dfsourcingrule.drop(['Inventory Org Name'], axis = 1)
#print(dfsourcingrule.shape, 'ITEMS 1 SHAPE')
dfsourcingrule['Item'] = dfsourcingrule['Item'].astype(str)

dfsourcingrule = pd.merge(dfsourcingrule, ABCD, on='SKU DC', how='left')
dfsourcingrule.rename(columns = {'Zebra ABCD Rating':'ABCD RATING'}, inplace = True)

dfsourcingrule['PLANNING CATEGORY'] = np.where(dfsourcingrule['ABCD RATING'] == 'UNASSIGN','Obsolete',
                                      np.where(dfsourcingrule['ABCD RATING'] == 'A','Super Core',
                                      np.where(dfsourcingrule['ABCD RATING'] == 'B','Core',
                                      np.where(dfsourcingrule['ABCD RATING'] == 'C','Non Core',
                                      np.where(dfsourcingrule['ABCD RATING'] == 'D','Slow Moving','NA' )))))

dfsourcingrule.drop(labels=['Item_y'], axis=1, inplace = True)
dfsourcingrule.rename(columns={'Item_x': 'Item'},inplace =True)

dfITEMS2 = dfsourcingrule[['Item','Sourcing Rule Name','Supplier Name','Supplier Number','Supplier Site Code','Product Family Description']]
#dfITEMS2 = dfsourcingrule[['Item','Sourcing Rule Name','Supplier Name','Supplier Number','Supplier Site Code']]
print(dfITEMS2.shape, 'ALL ITEMS 2')

(230428, 6) ALL ITEMS 2


In [19]:
dfITEMS21=dfITEMS2.rename(columns = {'Product Family Description':'Product_Family_Description'})
dfITEMS21.head()

,Item,Sourcing Rule Name,Supplier Name,Supplier Number,Supplier Site Code,Product_Family_Description
0,001.19RACKSIM,Transfer From SVC TX5.,NaN,NaN,NaN,GSD - FRU
1,00251-000,Global AQ Parkoprint AB,AQ Parkoprint AB,10982.0,Gavle-ZTE,SPS Parts
2,00255-000,Global AQ Parkoprint AB,AQ Parkoprint AB,10982.0,Gavle-ZTE,SPS Parts
3,00272-080,Global AQ Parkoprint AB,AQ Parkoprint AB,10982.0,Gavle-ZTE,SPS Parts
4,00272-080,Transfer from TX2,AQ Parkoprint AB,10982.0,Gavle-ZTC,SPS Parts


In [37]:
dfITEMS3=dfITEMS21[['Item','Product_Family_Description']]
#subset NA PN and convert those to numbers
dfITEMS3=dfITEMS3[dfITEMS3['Product_Family_Description'].isnull()]
print(dfITEMS3.shape, 'ITEMS 2 with NA SHAPE')
dfITEMS3.head()

(453, 2) ITEMS 2 with NA SHAPE


,Item,Product_Family_Description
56,100851093,NaN
57,1009304004,NaN
58,1009340001,NaN
59,1009341001,NaN
60,1013011001,NaN


In [40]:
dfITEMS3 = dfITEMS3[(dfITEMS3['Item'] != 'nan')]
dfITEMS3.shape

(206, 2)

In [42]:
#CONVERT TO NUMBER ITEM 
dfITEMS3['Item'] = dfITEMS3['Item'].astype(float)
dfITEMS3.head()

ValueError: could not convert string to float: '2605-10-01 00:00:00'

In [46]:
b=pd.to_numeric(dfITEMS3['Item'], errors='coerce').fillna(dfITEMS3['Item'])
b

56        1.00851e+08
57         1.0093e+09
58        1.00934e+09
59        1.00934e+09
60        1.01301e+09
             ...     
207520    6.08845e+10
207521    6.08845e+11
207777    7.53051e+15
207778    7.53051e+15
207780    7.53051e+15
Name: Item, Length: 206, dtype: object

In [11]:
dfITEMS21.Item.unique()

array(['nan'], dtype=object)

In [17]:
#CONVERT TO NUMBER ITEM 
dfITEMS2['Item'] = dfITEMS2['Item'].astype(int)
#merge again vs ABCD
dfITEMS2 = pd.merge(dfITEMS2, ABCD, on ='Item', how='left')

'Sourcing Rules WK 19.xlsx'

In [4]:
ABCD.head()

,Item,Zebra ABCD Rating,Product Family Description,SKU DC
0,.18523732.00,C,GSD - FRU,.18523732.00NL1
1,00017-000,NaN,GSD - FRU,00017-000NL1
2,0001AB,UNASSIGN,VC50XX,0001ABNL1
3,0003G,E,Tooling,0003GNL1
4,00040,UNASSIGN,EVM NON-SALEABLE,00040NL1


In [47]:
#dfglobal.to_excel('df.xlsx',index= False,sheet_name='Global consolidated DnS SV+ZCM')
with pd.ExcelWriter('VARIOUS WK7.xlsx') as writer:  
    dfITEMS3.to_excel(writer, sheet_name='ITEMS2',index= False)
    b.to_excel(writer, sheet_name='ABCD',index= False)

In [ ]:
#convert blanks to number and then merge again items

# Current CRM

In [3]:
#dfglobal.to_excel('df.xlsx',index= False,sheet_name='Global consolidated DnS SV+ZCM')
with pd.ExcelWriter(b) as writer:  
    #dfASN.to_excel(writer, sheet_name='CRM',index= False)
    dfSRDC.to_excel(writer, sheet_name='ASN SRDC',index= False)
    OHPivot.to_excel(writer, sheet_name = 'OH', index = False)
    dfsourcingrule.to_excel(writer, sheet_name='ITEMS1',index= False)
    #ABCD.to_excel(writer, sheet_name='ABCD',index= False)
d=(time.time() - start_time)/60
print("--- %s minutes to create VARIOUS WK file" % d)
#print(b)
c=fnmatch.filter(os.listdir('.'), 'Sourcing Rules*')
c=c[0] 
#print(c)

--- 120.16326831976572 minutes to create VARIOUS WK file


# Delete all duplicates

In [3]:
#MOVE SHEETS FROM ASN file
import xlwings as xw
a=fnmatch.filter(os.listdir('.'), 'ASNS*')
a=a[0]
#print(a)
#print(b)
pathtc = "C:\\Users\\MJ7255\\Desktop\\CRM\\" + a
pathf = "C:\\Users\\MJ7255\\Desktop\\CRM\\VARIOUS WK.xlsx"
wbtc = xw.Book(pathtc)
wbf = xw.Book(pathf)
wstc = wbtc.sheets(1)

wstc.api.Copy(After=wbf.sheets(1).api)
wbf.save()
wbtc.app.quit()

In [5]:
#print(ABCD.shape,'ABCD SHAPE DUPLICATE ITEMS')
ABCD=ABCD.drop_duplicates(subset='Item',keep='first') 
#print(ABCD.shape,'ABCD SHAPE NO DUPLICATE ITEMS')
dfITEMS2["Material Number"]=dfITEMS2["Item"]
dfITEMS2 = pd.merge(dfITEMS2, ABCD, on = 'Item', how = 'left')
print(dfITEMS2.shape, 'ITEMS2 after merge')
dfITEMS2.drop(labels = ['SKU DC','Zebra ABCD Rating'],axis=1, inplace = True)
dfITEMS2.insert(6, 'Business', 'Zebra')
dfITEMS2.insert(6, 'New Line Master Cap Model', '')
dfITEMS2.insert(6, 'Master file  ', '')
dfITEMS2.insert(6, 'Family', '')
dfITEMS2.insert(6, 'Value Stream', '')
dfITEMS2.insert(6, 'Ship Check', '')
dfITEMS2.insert(6, 'Sub Assemblies', '')
dfITEMS2.insert(6, 'Cost', 'Product')

(169702, 9) ITEMS2 after merge


# Delete NAN FROM ITEMS 2

In [36]:
dfITEMS2['Item'] = np.where(dfITEMS2['Item'] == 'nan', '', dfITEMS2['Item'])

dfITEMS2['a'] = pd.notnull(dfITEMS2['Item'])
dfITEMS21 = dfITEMS2[dfITEMS2['a'] == True]

print(dfITEMS21.shape)
dfITEMS21.head()

(192430, 6)


,Item,Sourcing Rule Name,Supplier Name,Supplier Number,Supplier Site Code,a
0,001.19RACKSIM,Transfer From SVC TX3 to NL1,NaN,NaN,NaN,True
1,00251-000,Global AQ Parkoprint AB,AQ Parkoprint AB,10982.0,Gavle-ZTE,True
2,00255-000,Global AQ Parkoprint AB,AQ Parkoprint AB,10982.0,Gavle-ZTE,True
3,00272-080,Global AQ Parkoprint AB,AQ Parkoprint AB,10982.0,Gavle-ZTE,True
4,00272-080,Transfer from TX2,AQ Parkoprint AB,10982.0,Gavle-ZTC,True


# NO

In [12]:
dfsourcingrule.loc[dfsourcingrule['Item'] == "282322"]

,Org,Inventory Org Name,Item,SKU DC,Sourcing Rule Name,Supplier Name,Supplier Number,Supplier Site Code,Item Status,TRANSFERS
19589,NL1,NL1-ZTE NL Inv Org,282322,282322NL1,Transfer From SVC TX3 to NL1,NaN,NaN,NaN,Zebra Inactive,YES


In [3]:
#ASN
a=fnmatch.filter(os.listdir('.'), 'ASNS*')
a=a[0] 
dfASN = pd.read_excel(a,dtype={'*Zebra PN': str},sheet_name = 'CRM',parse_dates=['*Planned Delivery Date.1'])
dfASN['*Planned Delivery Date.1']= dfASN['*Planned Delivery Date.1'].dt.strftime('%m-%d-%Y')

with pd.ExcelWriter('VARIOUS WK.xlsx') as writer:  
    dfASN.to_excel(writer, sheet_name='ASN SRDC',index= False)

In [5]:
ttry=("%s.xls" % strings)
ttry

'VARIOUS WK24.xls'

In [7]:
#dfglobal.to_excel('df.xlsx',index= False,sheet_name='Global consolidated DnS SV+ZCM')
with pd.ExcelWriter(ttry) as writer:  
    #dfASN.to_excel(writer, sheet_name='CRM',index= False)
    #dfSRDC.to_excel(writer, sheet_name='ASN SRDC',index= False)
    OHPivot.to_excel(writer, sheet_name = 'OH', index = False)
    #dfsourcingrule.to_excel(writer, sheet_name='ITEMS1',index= False)
    #ABCD.to_excel(writer, sheet_name='ABCD',index= False)
d=(time.time() - start_time)/60
print("--- %s minutes to create VARIOUS WK file" % d)
#print(b)
c=fnmatch.filter(os.listdir('.'), 'Sourcing Rules*')
c=c[0] 
#print(c)

--- 315.9642132639885 minutes to create VARIOUS WK file
